#  Neuralne mreže

**"A computing system made up of a number of simple, highly interconnected processing elements, which process information by their dynamic state response to external inputs"**



# Pregled
* Neuron
* Arhitektura 
* Forward propagation i backpropagation
* Odabir aktivacione funkcije
* Klasifikacija neuralnih mreža
  * Multilayer Perceptron (MLP)

## Neuralne mreže
* Model izračunavanja koji nam omogućava da definišemo kompleksnije hipoteze nego do sad
* Jedna istrenirana neuralna mreža = jedna hipoteza
  * Istrenirana = pronađene adekvatne vrednosti parametara (kao i do sad)
  * I dalje ćemo imati ulaz u vidu feature vektora $x$ 
  * I dalje ćemo imati izaz u vidu rezultata hipoteze $h_w(x)$
* Sastoje se od jednostavnih jedinica - **neurona**

## Neuron
* Jedan neuron je zapravo trojka $(w, b, f)$ - $w$ je vektor težina, $b$ je *bias* vrednost, a $f$ je **aktivaciona funkcija**
* Svaki neuron dobija ulaz (vektor $x$) i transformiše ga po formuli $f(wx+b)$ - linearna kombinacija sa biasom nad kojom je primenjena funkcija aktivacije
* 1 neuron sa sigmoidnom aktivacijom je u potpunosti ekvivalentan logističkoj regresiji!
* Napomena: Neki autori umesto odvojene vrednosti za bias dodaju još jednu vrednost u vektor težina i na svaki feature vektor dodaju novi feature čija je vrednost uvek jedan - ovo daje iste rezultate ali menja notaciju


## Arhitektura
* Sada kada znamo kako radi jedan neuron, neuralna mreža je samo kombinacija velikog broja njih
* Neuroni se kombinuju "u dve dimenzije"
* Prvo "vertikalno", gde više neurona čini jedan **sloj** mreže (više transformacija odjednom tako da se od feature vektora $x$ dobije novi feature vektor $x'$)
  * Kombinujemo parametre neurona u parametre sloja tako da dobijamo $W$ - matricu težina, $b$ - jedinstvenu bias vrednost za ceo sloj, $f$ - jedinstvenu funkciju aktivacije za ceo sloj
* Zatim "horizontalno", gde se više slojeva niže tako da izlaz jednog postaje ulaz narednog
* Ulazni podaci čine **ulazni sloj**, poslednji sloj mreže čini **izlazni sloj**, a svi slojevi između se zovu **skriveni slojevi** (jer njihove vrednosti ne posmatramo direktno)
* Dakle, uloga svakog sloja je da transformiše feature vektor u novi feature vektor (osim izlaznog, koji će uglavnom da vrši finalna izračunavanja - npr. softmax)


* **Primer**: mreža za binarnu klasifikaciju feature vektora dimenzije 10 sa jednim skrivenim slojem sastavljenim od 3 neurona
  * Skicirati mrežu, definisati sve njene parametre, kao i hipotezu
  * $a^{(1)} = f^{(1)}(z^{(1)}) = f^{(1)}(W^{(1)}x+b^{(1)})$
  * $h = a^{(2)} = f^{(2)}(z^{(2)}) = f^{(2)}(W^{(2)}a^{(1)}+b^{(2)})$
  
## Forward propagation i backpropagation
* Proces izračunavanja kojim mreža dobija hipotezu od ulaznih feature-a naziva se **forward propagation** (propagacija unapred)
* Do sada nismo pomenuli kako treniramo mrežu tj. kako tokom treninga ažuriramo parametre
* Upravo to radi **backpropagation** (propagacija unazad) - proces koji nakon propagacije unapred računa funkciju troška i propagira njene izvode unazad kroz mrežu, koristeći ih za ažuriranje parametara na svim slojevima
  * Detalje ćemo sada preskočiti, ali ovo je ključan mehanizam neuralnih mreža i važno je razumeti kako funkcioniše

## Odabir aktivacione funkcije
* Pojam aktivacione funkcije smo videli još u primeru logističke regresije (ali tada nismo koristili ovaj naziv)
* Svrha aktivacione funkcije je da unese nelinearnost u sistem (bez nje bi mreža sa $N$ slojeva bila ekvivalentna mreži sa jednim slojem, pa ne koristimo punu moć arhitekture)
* Kandidati:
  * identitet - $f(z) = z$
  * sigmoid - $f(z) = \frac{1}{1+e^{(-z)}}$
  * tangens hiperbolički - $f(z) = tanh(z)$
  * rectified linear unit (ReLU) - $f(z) = max(0, z)$
  * leaky ReLU - $f(z) = z$ ako je $z>=0$, a $f(z)=\alpha z$ u suprotnom, gde je $\alpha$ mala unapred određena konstanta

## Klasifikacija neuralnih mreža
- Neuralne mreže koje nemaju cikluse u grafu povezanosti neurona nazivamo **feedforward mreže** (suprotno, one koje imaju cikluse zovemo **rekurentne mreže**)
- Mreže koje smo do sada definisali su isključivo feedforward
- Najprostija feedforward mreža je **perceptron**: 0 skrivenih slojeva
- Ukoliko imamo 1 ili više skrivenih slojeva takvu mrežu zovemo **multilayer perceptron (MLP)**
  - MLP ćemo implementirati za MNIST problem na tri nivoa apstrakcije: 
    - Prvo, koristeći samo numpy, dakle bez specijalizovanih biblioteka
    - Zatim, koristeći TensorFlow
    - Zatim, koristeći Keras

## Dodatni resursi

http://cs231n.github.io/neural-networks-1/


http://neuralnetworksanddeeplearning.com/


http://karpathy.github.io/neuralnets/


http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks/

## Implementacija MLP uz numpy bez TensorFlow
[Link](https://github.com/rand0musername/psiml2017-workshops/blob/master/2%20-%20NN/code/nn.py)

## Implementacija MLP u TensorFlow

In [0]:
# TODO

## Implementacija MLP u biblioteci Keras

In [0]:
# TODO